In [1]:
import torch
import pyro
from torch.utils.data import DataLoader, random_split, TensorDataset
import numpy as np
import os
from cities.utils.data_grabber import find_repo_root
from cities.utils.data_loader import ZoningDataset
from cities.modeling.evaluation import prep_data_for_test, test_performance

from cities.modeling.simple_linear import SimpleLinear
from cities.modeling.svi_inference import run_svi_inference
from pyro.infer import Predictive
from chirho.robust.handlers.predictive import PredictiveModel



root = find_repo_root()

In [38]:
to_map = torch.tensor([0,10, 20, 10, 0])
map = {0:0, 10:1, 20:2}
print([map[x.item()] for x in to_map])
#mapped = torch.tensor([map[x] for x in to_map])
#print(mapped)

[0, 1, 2, 1, 0]


In [44]:
train_loader, test_loader, categorical_levels = prep_data_for_test()

kwarg_names = {"categorical": ['limit_id', 'neighborhood_id'],
              "continuous": {'parcel_area'}, "outcome": 'housing_units'}

model_class = SimpleLinear
n_steps=600 
plot = True


assert all(item in kwarg_names.keys() for item in ["categorical", "continuous", "outcome"])
assert kwarg_names["outcome"] not in kwarg_names["continuous"]

train_data = next(iter(train_loader))
test_data = next(iter(test_loader))

_train_data = {}
_train_data['outcome'] = train_data['continuous'][kwarg_names['outcome']]
_train_data['categorical'] = {key: val for key, val in train_data['categorical'].items() if key in kwarg_names['categorical']}
_train_data['continuous'] = {key: val for key, val in train_data['continuous'].items() if key in kwarg_names['continuous']}

_test_data = {}
_test_data['outcome'] = test_data['continuous'][kwarg_names['outcome']]
_test_data['categorical'] = {key: val for key, val in test_data['categorical'].items() if key in kwarg_names['categorical']}
_test_data['continuous'] = {key: val for key, val in test_data['continuous'].items() if key in kwarg_names['continuous']}

#_train_data['categorical'] = {key: val for key, val in train_data['categorical'].items() if key in kwarg_names['categorical']}
#_train_data['continuous'] = {key: val for key, val in train_data['continuous'].items() if key in kwarg_names['continuous']}
print([_test_data['categorical'][name].shape for name in kwarg_names["categorical"]])
print([_test_data['categorical'][name].unique() for name in kwarg_names["categorical"]])

#print(_train_data['outcome'])


##########################
# eliminate test categories not in the training data
##########################
def apply_mask(data, mask):
        return {key: val[mask] for key, val in data.items()}

mask = torch.ones(len(test_data['outcome']), dtype=torch.bool)
for key, value in test_data['categorical'].items():
    mask = mask *  torch.isin(test_data['categorical'][key],(train_data['categorical'][key].unique()))

test_data['categorical'] = apply_mask(test_data['categorical'], mask)
test_data['continuous'] = apply_mask(test_data['continuous'], mask)
test_data['outcome'] = test_data['outcome'][mask]

for key in test_data['categorical'].keys():
        assert test_data['categorical'][key].shape[0] == mask.sum()
for key in test_data['continuous'].keys():
    assert test_data['continuous'][key].shape[0] == mask.sum()

# raise error if sum(mask) < .5 * len(test_data['outcome'])
if sum(mask) < .5 * len(test_data['outcome']):
    raise ValueError("Sampled test data has too many new categorical levels, consider decreasing train size")


mappings = {}
for name in _train_data['categorical'].keys():
    print(name)
    print("original", _test_data['categorical'][name])
    unique_train = torch.unique(_train_data['categorical'][name])
    mappings[name] = {v.item(): i for i, v in enumerate(unique_train)}
    print(mappings[name])
    _train_data['categorical'][name] = torch.tensor([mappings[name][x.item()] for x in  _train_data['categorical'][name]])
    _test_data['categorical'][name] = torch.tensor([mappings[name][x.item()] for x in  _test_data['categorical'][name]])
    print("recoded", _test_data['categorical'][name])


[torch.Size([61]), torch.Size([61])]
[tensor([0, 1, 2]), tensor([ 0,  2,  3,  5,  6,  7,  8, 10, 11, 12, 16, 17, 18, 21, 22, 24, 26, 30,
        32, 33, 37, 38, 40, 42, 52, 58, 59, 62, 64])]
neighborhood_id
original tensor([ 6, 32, 21,  3,  8, 11, 32,  5,  7,  6, 42, 30,  7,  3,  7, 18,  2, 38,
        59, 17, 10,  8, 37,  0,  7, 22,  7,  8, 12, 18, 16,  7,  0, 52, 42,  7,
        32, 32,  7, 64,  8, 16, 62, 33,  3, 32, 11, 12,  3, 42, 17, 12, 58,  7,
         2, 17, 24, 26, 32, 40,  3])
{0: 0, 1: 1, 2: 2, 3: 3, 4: 4, 6: 5, 7: 6, 8: 7, 9: 8, 10: 9, 11: 10, 12: 11, 13: 12, 14: 13, 15: 14, 17: 15, 18: 16, 19: 17, 20: 18, 21: 19, 22: 20, 23: 21, 24: 22, 25: 23, 27: 24, 28: 25, 29: 26, 30: 27, 31: 28, 32: 29, 33: 30, 34: 31, 35: 32, 36: 33, 37: 34, 38: 35, 39: 36, 40: 37, 41: 38, 42: 39, 43: 40, 44: 41, 45: 42, 46: 43, 47: 44, 48: 45, 49: 46, 50: 47, 51: 48, 53: 49, 54: 50, 55: 51, 56: 52, 57: 53, 60: 54, 61: 55, 62: 56, 63: 57}


KeyError: 5

In [ ]:

test_performance(SimpleLinear, kwarg_names, train_loader,
                  test_loader, categorical_levels)


